In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [35]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily = TavilySearchResults(include_images=True)

In [36]:
result = tavily.invoke("what is large language model")

In [37]:
result

[{'title': 'Large language model - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/Large_language_model',
  'content': 'A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.',
  'score': 0.9536608},
 {'title': 'What are Large Language Models? | A Comprehensive LLMs Guide',
  'url': 'https://www.elastic.co/what-is/large-language-models',
  'content': 'What are large language models (LLMs)?\nExperiment with RAG using AI Playground\nLarge language model definition\nA large language model (LLM) is a deep learning algorithm that can perform a variety of natural language processing (NLP) tasks. Large language models use transformer models and are trained using massive datasets — hence, large. This enables them to recognize, translate, predict, or generate text or other content. [...] H

In [38]:
result[0]['content']

'A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.'

In [39]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.tools import WikipediaQueryRun

In [40]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.utilities import WikipediaAPIWrapper

In [41]:
wiki_api = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=200)
wikipedia = WikipediaQueryRun(api_wrapper=wiki_api)

In [42]:
arxiv_api = ArxivAPIWrapper(top_k_results=3, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api)

In [43]:
# tools = [wikipedia, arxiv, tavily]
tools = [tavily]

In [44]:
from langchain_groq import ChatGroq

llm = ChatGroq(model='qwen-qwq-32b')

In [45]:
llm_with_tools = llm.bind_tools(tools=tools)

In [46]:
result = llm_with_tools.invoke("explain the terrif war between the China and the America")

In [47]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xyac', 'function': {'arguments': '{"query": "US China terrif war explanation"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 444, 'prompt_tokens': 198, 'total_tokens': 642, 'completion_time': 1.094418831, 'prompt_time': 0.018208344, 'queue_time': 0.247361781, 'total_time': 1.112627175}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_fbb7e6cc39', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3c5cd676-d188-47da-b2e3-13c4a8daf041-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'US China terrif war explanation'}, 'id': 'call_xyac', 'type': 'tool_call'}], usage_metadata={'input_tokens': 198, 'output_tokens': 444, 'total_tokens': 642})

In [49]:
tool_calls = result.tool_calls
tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'US China terrif war explanation'},
  'id': 'call_xyac',
  'type': 'tool_call'}]

In [50]:
# wikipedia.invoke(tool_calls[0]['args'])
tavily.invoke(tool_calls[0]['args'])

[{'title': 'China–United States trade war - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/China%E2%80%93United_States_trade_war',
  'content': 'An economic conflict between China and the United States has been ongoing since January 2018, when U.S. President Donald Trump began setting tariffs and other trade barriers on China with the goal of forcing it to make changes to what the U.S. says are longstanding unfair trade practices and intellectual property theft.[1] The first Trump administration stated that these practices may contribute to the U.S.–China trade deficit, and that the Chinese government requires transfer of American [...] ^ Tan, Huileng; Mody, Seema (April 4, 2018). "China\'s ambassador to the US explains why the country is striking back". CNBC. Archived from the original on May 13, 2019. Retrieved May 12, 2019.\n^ "China imposes additional tariffs on U.S. products worth 50 bln USD". www.xinhuanet.com. Archived from the original on April 4, 2018. Retrieved April 4, 

In [58]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage

In [59]:
class State(TypedDict):
    messages : Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [60]:
class LLMNode:
    def __init__(self, llm):
        self.llm = llm
    def __call__(self, state:State):
        return {"messages": [self.llm.invoke(state['messages'])]}
    
llm_node = LLMNode(llm_with_tools)

In [61]:
from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tools)

In [62]:
graph_builder.add_node("llm", llm_node)
graph_builder.add_node("tools", tool_node)

In [63]:
from langgraph.graph import START
from langgraph.prebuilt import tools_condition

graph_builder.add_edge(START, "llm")
graph_builder.add_conditional_edges("llm", tools_condition)
graph_builder.add_edge("tools", "llm")

In [64]:
agent = graph_builder.compile()

In [65]:
from IPython.display import Image, display
display(Image(agent.get_graph().draw_mermaid_png()))

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [66]:
messages = agent.invoke({"messages":HumanMessage(content="explain the terrif war between the China and the America")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

explain the terrif war between the China and the America
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_jvng)
 Call ID: call_jvng
  Args:
    query: US-China trade war current status and key issues
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "China–United States trade war - Wikipedia", "url": "https://en.wikipedia.org/wiki/China%E2%80%93United_States_trade_war", "content": "An economic conflict between China and the United States has been ongoing since January 2018, when U.S. President Donald Trump began setting tariffs and other trade barriers on China with the goal of forcing it to make changes to what the U.S. says are longstanding unfair trade practices and intellectual property theft.[1] The first Trump administration stated that these prac

In [67]:
messages['messages'][3].pretty_print()

================================== Ai Message ==================================

The U.S.-China trade war, which began in 2018, is a complex, ongoing economic and geopolitical conflict rooted in disputes over trade imbalances, intellectual property theft, forced technology transfers, and China's industrial policies. Here's a concise explanation:

### **Origins (2018–2019):**
- **Triggers:** U.S. President Donald Trump accused China of unfair trade practices, including subsidizing state-owned enterprises, stealing intellectual property, and pressuring foreign companies to hand over technology via joint ventures.
- **Actions:** The U.S. imposed tariffs on over $360 billion in Chinese imports (steel, electronics, consumer goods), while China retaliated with tariffs on $110 billion of U.S. goods (soybeans, aircraft). This sparked a cycle of "tit-for-tat" trade restrictions.

### **Key Issues:**
1. **Economic Disputes:**  
   - The U.S. sought to reduce its massive bilateral trade deficit 